In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import openpyxl
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.drawing.image import Image
import win32com.client as win32
import os
from PIL import ImageGrab
import win32com.client

olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

In [94]:
query = """
select "sliceDate","sellerId","sellerName","customerName","materialName","netPrice","currency","demandQuantity","netValue","plannedShipmentDate","plannedInvoiceDate" , "orderPositionStatus","orderStatus","orderStatusProgress"
from xxwh."sapDealsInternalMarket" 
where "sliceDate" = (select max("sliceDate") as max_data from xxwh."sapDealsInternalMarket")
"""
data=export_from_WHWEEK(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:168: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  def export_from_WHWEEK(query):


In [81]:
query = """
select * from xxwh."sapDealsInternalMarket" 
where "sliceDate" = (select max("sliceDate") as max_data from xxwh."sapDealsInternalMarket")
"""
data=export_from_WHWEEK(query)
data.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:168: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  def export_from_WHWEEK(query):


,##batch_id,##deleted,##inserted,##origin_name,sliceDate,contractId,orderId,orderPositionId,invoiceId,orderPositionStatus,...,plannedInvoiceDate,creationInvoiceDate,accountingInvoiceDate,invoiceStatus,incoterms,incoterms_2,paymentConditions,contractCreationDate,contractNumber,transportType
0,36570039@WHWEEK,N,2024-05-22 00:00:02,WHFLEX,2024-05-21,4000105555,0000102605,10,0092280969,completed,...,2020-01-01,2020-01-29,2020-01-23,C,NDT,"Большая, 310",C030,2020-01-14,ПД-А-69-19-02299,None
1,36570039@WHWEEK,N,2024-05-22 00:00:02,WHFLEX,2024-05-21,4000106135,0000102665,10,0092279448,completed,...,2020-01-01,2020-01-28,2020-01-17,C,EXW,г.Омск,A000,2020-01-20,СУЭК-А/20-006Е,None


In [95]:
data.orderPositionStatus.unique()
# orderPositionStatus	orderStatus	orderStatusProgress
data.iloc[:, 3:]
data

,sliceDate,sellerId,sellerName,customerName,materialName,netPrice,currency,demandQuantity,netValue,plannedShipmentDate,plannedInvoiceDate,orderPositionStatus,orderStatus,orderStatusProgress
0,2024-05-21,Y100,"АО ""СГК - Новосибирск""","АО ""РЭС""",УГОЛЬ ТЕХНИЧЕСКИЙ,1425.31,RUB,35.00,49885.85,2020-01-01,2020-01-01,completed,completed,1/1
1,2024-05-21,4900,"ООО ""АЛТАЙ-СУЭК""","ООО ""Стайер-99""",УГОЛЬ ДО (25-60) ОБОГАЩЕННЫЙ,2833333.33,RUB,59.00,167166.67,2020-01-01,2020-01-01,completed,completed,1/1
2,2024-05-21,5N00,"АО ""СУЭК-Кузбасс""","ООО ""СЦ ""СЭР""",УГОЛЬ ДР (0-200),1420.00,RUB,60.00,85200.00,2020-02-01,2020-01-01,completed,completed,1/1
3,2024-05-21,2D00,"АО ""Разрез Березовский""","МУП ""ПП ЖКХ-5"" П.СТРЕЛКА",УГОЛЬ 2БР (0-300),1080.00,RUB,1000.00,1080000.00,2020-01-01,2020-01-01,completed,completed,1/1
4,2024-05-21,6600,"ООО ""Хабаровск-СУЭК""","ООО ""РЕГИОНОПТ""",УГОЛЬ 2БОМС (5-40),3983.33,RUB,100.00,398333.00,2020-01-01,2020-01-01,completed,completed,4/4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50303,2024-05-21,4F00,"ООО ""СУЭК-Хакасия""","ООО ""Стандарт-Уголь""",УГОЛЬ ДО (25-50),1.00,RUB,1.00,1.00,2024-01-01,2023-08-01,not_completed,not_completed,5/55
50304,2024-05-21,4F00,"ООО ""СУЭК-Хакасия""","ООО ""Стандарт-Уголь""",УГОЛЬ ДПК (50-200),1.00,RUB,1.00,1.00,2024-01-01,2023-08-01,not_completed,not_completed,5/55
50305,2024-05-21,4F00,"ООО ""СУЭК-Хакасия""","ООО ""Стандарт-Уголь""",УГОЛЬ ДПК (50-200),1.00,RUB,1.00,1.00,2024-01-01,2023-08-01,not_completed,not_completed,5/55
50306,2024-05-21,3D00,"АО ""Разрез Тугнуйский""","МУП ЖКХ ""Тепловик""",УГОЛЬ ДР (0-300),2740.00,RUB,1325.65,3632281.00,2024-02-01,2024-02-01,completed,not_completed,1/2


In [69]:
print_to_excel = True

In [105]:
data_work = data[data.orderStatus=='not_completed'].reset_index()
# Merge data
data_work['Comp_segment'] = merge_SalesUnits(data_work,'sellerId',merge_col='ocpSegment')
data_work['Cpty_id'] = merge_Mapping(data_work,'customerName')
data_work['Cpty_segment'] = merge_SalesUnits(data_work,'Cpty_id',merge_col='ocpSegment')
data_work['Comp_holding'] = merge_SalesUnits(data_work,'sellerId',merge_col='holding')
data_work['Cpty_holding'] = merge_SalesUnits(data_work,'Cpty_id',merge_col='holding')
# Counterclaims
data_counterclaims = data_work[data_work.Cpty_id!='External']
# data_counterclaims_pivot = data_counterclaims.pivot_table(index=['Comp_segment','sellerId','Cpty_segment','Cpty_id','materialName','currency'], values=['demandQuantity','netValue'], aggfunc=sum).reset_index()
data_counterclaims_pivot = data_counterclaims.pivot_table(index=['Comp_segment','sellerId','Cpty_segment','Cpty_id','currency'], values=['demandQuantity','netValue'], aggfunc=sum).reset_index()
# data_counterclaims_pivot['Comp_Cpty'] = data_counterclaims_pivot[['sellerId','Cpty_id','materialName','currency']].astype(str).agg('.'.join, axis=1)
# data_counterclaims_pivot['Cpty_Comp'] = data_counterclaims_pivot[['Cpty_id','sellerId','materialName','currency']].astype(str).agg('.'.join, axis=1)
data_counterclaims_pivot['Comp_Cpty'] = data_counterclaims_pivot[['sellerId','Cpty_id','currency']].astype(str).agg('.'.join, axis=1)
data_counterclaims_pivot['Cpty_Comp'] = data_counterclaims_pivot[['Cpty_id','sellerId','currency']].astype(str).agg('.'.join, axis=1)
data_merge = data_counterclaims_pivot.merge(data_counterclaims_pivot, how='left', left_on='Comp_Cpty', right_on='Cpty_Comp')
# data_merge = data_merge.iloc[:,[0,1,2,3,4,5,6,7,16,17]]
data_merge = data_merge[['Comp_segment_x','sellerId_x','Cpty_segment_x','Cpty_id_x','currency_x','demandQuantity_x','netValue_x','demandQuantity_y','netValue_y']]
data_merge['Amount_is_equal'] = data_merge.netValue_x==data_merge.netValue_y
data_counterclaims_final = data_merge.sort_values(['demandQuantity_y','sellerId_x','Cpty_id_x'])
# manual map
manual_map = data_work.loc[data_work.Comp_segment=='External',['sellerId','sellerName','Comp_segment','Comp_holding']].drop_duplicates()
# Empty data
data_empty = data_work[data_work.isna().any(axis=1)]
data_empty_sellerName = data_empty[data_empty.sellerName.isna()]
assert data_empty.shape ==data_empty_sellerName.shape
data_empty_final = data_empty_sellerName[['sellerId','sellerName']].drop_duplicates()

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  def export_from_RISKCUSTOM(query):
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SUEK RU' 'SUEK RU' 'SUEK RU' ... 'SUEK RU' 'SUEK RU' 'SUEK RU']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data['oebs12ShortCode_merge'] = df.merge(data_oebs12ShortCode, how='left', left_on=col, right_on='oebs12ShortCode', validate='many_to_one').iloc[:, -1].fillna('External')
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or 

In [104]:
data_counterclaims_final
data.columns

,Comp_segment_x,sellerId_x,Cpty_segment_x,Cpty_id_x,currency_x,demandQuantity_x,netValue_x,demandQuantity_y,netValue_y,Amount_is_equal
10,SUEK RU,2100,SUEK RU,2100,RUB,4.000000e+01,2.360000e+04,40.0,23600.0,True
0,External,1101,SGC,U100,RUB,3.515429e+06,5.517598e+09,NaN,NaN,False
11,SUEK RU,2100,SUEK RU,2400,RUB,1.619000e+03,1.405425e+06,NaN,NaN,False
12,SUEK RU,2100,SUEK RU,4900,RUB,9.468000e+03,2.033262e+07,NaN,NaN,False
4,SUEK RU,2100,SGC,V100,RUB,5.509637e+06,3.823330e+09,NaN,NaN,False
5,SUEK RU,2100,SGC,V200,RUB,2.600080e+05,2.506477e+08,NaN,NaN,False
6,SUEK RU,2100,SGC,V300,RUB,3.314247e+06,3.934949e+09,NaN,NaN,False
7,SUEK RU,2100,SGC,VF00,RUB,5.400000e+04,5.436605e+07,NaN,NaN,False
8,SUEK RU,2100,SGC,VG00,RUB,1.072335e+04,1.017869e+07,NaN,NaN,False
9,SUEK RU,2100,SGC,VH00,RUB,3.030230e+05,3.585292e+08,NaN,NaN,False


In [107]:
if print_to_excel == True:
    Output_file = f'{str(date.today())}_xxwh-sapDealsInternalMarket.xlsx'
    manual_map.to_excel(Output_file, sheet_name='Manual_map', index=False)
    new_list(data_counterclaims_final,Output_file,'Counterclaims')
    new_list(data_work,Output_file,'Data')
    new_list(data_empty_final,Output_file,'Empty_data')